# Customer Alliance Hotel Ranking Project


### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Importing Data

In [2]:
# importing data while parsing 'crawled_at' column as datetime object

data_df = pd.read_csv('TA_CityRank_Hack_DD_pull_3.csv', sep=';', low_memory=False, skipinitialspace=True, error_bad_lines=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
     print(data_df.head(10))

   hotel_id              hotel_name            street zip_code    city  \
0        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
1        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
2        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
3        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
4        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
5        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
6        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
7        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
8        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   
9        50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  Berlin   

  country  crawled_at  review_count_state  average_rating_state  \
0      DE  2015-08-19                 270                  80.0   
1      DE  2015-08-20                 270          

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776337 entries, 0 to 776336
Data columns (total 61 columns):
hotel_id                                 776337 non-null int64
hotel_name                               776337 non-null object
street                                   776305 non-null object
zip_code                                 776337 non-null object
city                                     776337 non-null object
country                                  776337 non-null object
crawled_at                               776337 non-null object
review_count_state                       776337 non-null int64
average_rating_state                     776337 non-null float64
sleep_rating_state                       133359 non-null float64
location_rating_state                    133376 non-null float64
room_rating_state                        133202 non-null float64
service_rating_state                     133376 non-null float64
value_rating_state                       133145 non-nu

In [4]:
data_df.describe()

,hotel_id,review_count_state,average_rating_state,sleep_rating_state,location_rating_state,room_rating_state,service_rating_state,value_rating_state,clean_rating_state,city_rank,...,value_rating_positive_count_todate,value_rating_neutral_count_todate,value_rating_negative_count_todate,value_rating_count_todate,value_rating_normalized_todate,clean_rating_positive_count_todate,clean_rating_neutral_count_todate,clean_rating_negative_count_todate,clean_rating_count_todate,clean_rating_normalized_todate
count,776337.000000,776337.000000,776337.000000,133359.000000,133376.000000,133202.000000,133376.000000,133145.000000,133234.000000,734966.000000,...,173180.000000,173180.000000,173180.000000,173180.000000,173180.000000,173180.000000,173180.000000,173180.000000,173180.000000,173180.000000
mean,90389.099417,562.764177,80.782250,82.152311,87.564854,78.360385,82.226562,80.602276,85.899995,461.595210,...,0.320176,0.068564,0.010359,0.399099,25.088783,0.341466,0.038457,0.006161,0.386084,25.760947
std,86677.161277,848.583046,9.340559,8.525900,9.669628,10.269200,8.958460,8.169351,9.047269,667.303172,...,0.670150,0.279408,0.106369,0.771805,39.492981,0.709165,0.208658,0.079931,0.765275,41.214509
min,19.000000,1.000000,20.000000,20.000000,50.000000,20.000000,40.000000,40.000000,40.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25145.000000,148.000000,80.000000,80.000000,80.000000,70.000000,80.000000,80.000000,80.000000,111.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,42661.000000,334.000000,80.000000,80.000000,90.000000,80.000000,80.000000,80.000000,90.000000,274.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,137741.000000,692.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,587.000000,...,1.000000,0.000000,0.000000,1.000000,60.000000,1.000000,0.000000,0.000000,1.000000,80.000000
max,352001.000000,34587.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,13232.000000,...,15.000000,6.000000,5.000000,21.000000,100.000000,15.000000,5.000000,2.000000,17.000000,100.000000


### Data Integrity Check and EDA

In [5]:
# How many hotels before dropping rows where city_rank (response) is NA? 1478 hotels. There are 16 more unique hotel_ids than
# there are hotel names

data_df['hotel_id']=data_df['hotel_id'].astype(str)
print(data_df[['hotel_id','hotel_name']].nunique())

hotel_id      1478
hotel_name    1462
dtype: int64


In [6]:
# How many hotels after dropping rows where city_rank (response) is NA? 1456, so we lose 22 hotels which have no useful data. 
# Again there are fewer hotel names as there are ids

print(data_df[data_df['city_rank'].notna()].shape)
print(data_df[data_df['city_rank'].notna()][['hotel_id','hotel_name']].nunique())

(734966, 61)
hotel_id      1456
hotel_name    1440
dtype: int64


In [7]:
# storing the rows of the nan city_rank out of main dataframe. 41,371 rows linked to 978 hotels

data_missing_df=data_df[data_df['city_rank'].isna()]
print(data_missing_df.shape)
print(data_missing_df[['hotel_id','hotel_name']].nunique())

(41371, 61)
hotel_id      978
hotel_name    976
dtype: int64


In [8]:
# storing the rows with a non-null value for city_rank in another dataframe

data_clean_df=data_df[data_df['city_rank'].notna()]

In [9]:
# storing the list of the 44 hotels with no useable data 

data_missing_hotels_id = set(data_df['hotel_id'])-set(data_clean_df['hotel_id'])
data_missing_hotels_name = set(data_df['hotel_name'])-set(data_clean_df['hotel_name'])

In [10]:
data_missing_hotels_id

{'114341',
 '176681',
 '176691',
 '176701',
 '176711',
 '236571',
 '268841',
 '268851',
 '268861',
 '268871',
 '268891',
 '268911',
 '268941',
 '319381',
 '322581',
 '324671',
 '49621',
 '49751',
 '49761',
 '49771',
 '49781',
 '49791'}

In [11]:
data_missing_hotels_name

{'Airline Moulinsart',
 'Alter Krug',
 'Cafe am neuen See',
 'Germania Airlines',
 'Grands Augustins/2154',
 'INDUSTRY STANDARD',
 'Loretta am Wannsee',
 'Luise',
 'Orderbird',
 'Prater Garten Berlin',
 'Restaurant Horváth',
 'Schleusenkrug',
 'Small Planet Airlines',
 'Sweet Inn BERTHE I',
 'Sweet Inn BOETIE IV',
 'Sweet Inn BRANCION',
 'Sweet Inn RAVIGNAN I',
 'Sweet Inn RAVIGNAN II',
 'Sweet Inn RIVOLI II',
 'Sweet Inn ROQUETTE III',
 'TXOKOA',
 'Wirtshaus Max und Moritz'}

In [12]:
# examining the rows of an example of these hotels. 

data_missing_example_df = data_df[data_df['hotel_id'] == '114341']

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
     print(data_missing_example_df.head())

       hotel_id hotel_name                       street zip_code    city  \
461409   114341  Orderbird  Ritterstraße 12-14, Aufg. 4    10969  Berlin   
461410   114341  Orderbird  Ritterstraße 12-14, Aufg. 4    10969  Berlin   
461411   114341  Orderbird  Ritterstraße 12-14, Aufg. 4    10969  Berlin   
461412   114341  Orderbird  Ritterstraße 12-14, Aufg. 4    10969  Berlin   
461413   114341  Orderbird  Ritterstraße 12-14, Aufg. 4    10969  Berlin   

       country  crawled_at  review_count_state  average_rating_state  \
461409      DE  2016-06-01                 217                  80.0   
461410      DE  2016-06-02                 217                  80.0   
461411      DE  2016-06-03                 217                  80.0   
461412      DE  2016-06-04                 217                  80.0   
461413      DE  2016-06-05                 217                  80.0   

        sleep_rating_state  location_rating_state  room_rating_state  \
461409                 NaN            

### Segmenting the clean data into Berlin and Paris

In [14]:
print(data_clean_df['city'].value_counts(),'\n')
print(data_clean_df['country'].value_counts(),'\n')    

Berlin                                       367427
Paris                                        307882
Paris 17                                       5731
Paris 06                                       5120
Paris 12                                       4698
Berlin - Charlottenburg                        4616
Paris 14                                       4611
Paris 16                                       3628
Paris 01                                       2740
Paris 07                                       2544
Paris 15                                       2111
Paris 09                                       1823
Paris 10                                       1562
Paris 03                                       1488
PARIS                                          1407
Berlin - Spandau                                933
Berlin-Schönefeld                               932
Paris - Tour Eiffel                             928
Berlin-Schmöckwitz                              928
Paris - 6. A

In [15]:
# converting city name to all lower case

data_clean_df['city']=data_clean_df['city'].str.lower()
print(data_clean_df['city'].value_counts(),'\n') 

C:\Users\Dina\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


berlin                                       367427
paris                                        309289
paris 17                                       5731
paris 06                                       5120
paris 12                                       4698
berlin - charlottenburg                        4616
paris 14                                       4611
paris 16                                       3628
paris 01                                       2740
paris 07                                       2544
paris 15                                       2111
paris 09                                       1823
paris 10                                       1562
paris 03                                       1488
berlin - spandau                                933
berlin-schönefeld                               932
paris - tour eiffel                             928
berlin-schmöckwitz                              928
paris - opera, st lazare, grands magasins       924
paris - 6. a

In [16]:
data_clean_df[(data_clean_df['country']=='DE')].shape

(380112, 61)

In [17]:
sum(data_clean_df['city'].str.contains('berlin'))

379747

In [18]:
data_clean_df[['city', 'country']].isnull().sum()

city       0
country    0
dtype: int64

In [19]:
# storing Berlin Germany data in a separate dataframe. Berlin is entered in a non-consistent way

berlin_df=data_clean_df[(data_clean_df['country']=='DE') & (data_clean_df['city'].str.contains('berlin')|
                                                            data_clean_df['city'].str.contains('berlín'))]
berlin_df.shape

(380088, 61)

In [20]:
# storing Paris France data in a separate dataframe

paris_df=data_clean_df[(data_clean_df['country']=='FR') & (data_clean_df['city'].str.contains('paris'))]
paris_df.shape

(354854, 61)

In [21]:
data_clean_df[(data_clean_df['country']=='DE') & (data_clean_df['city'].str.contains('paris'))].shape

(24, 61)

### Verifying features' definitions

In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
     print(berlin_df.head())

  hotel_id              hotel_name            street zip_code    city country  \
0       50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  berlin      DE   
1       50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  berlin      DE   
2       50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  berlin      DE   
3       50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  berlin      DE   
4       50  Bleibtreu Hotel Berlin  Bleibtreustr. 31    10707  berlin      DE   

   crawled_at  review_count_state  average_rating_state  sleep_rating_state  \
0  2015-08-19                 270                  80.0                80.0   
1  2015-08-20                 270                  80.0                80.0   
2  2015-08-21                 270                  80.0                80.0   
3  2015-08-22                 270                  80.0                80.0   
4  2015-08-23                 270                  80.0                80.0   

   location_rating_state  room_rating_

** Important Equations **

- response_rate_count_todate = response_rate_negative_count_todate + response_rate_neutral_count_todate+response_rate_positive_count_todate

- response_rate_todate = (response_rate_negative_todate + response_rate_neutral_todate + response_rate_positive_todate)/3 ?

-  average_rating_count_todate = average_rating_negative_count_todate + average_rating_neutral_count_todate + average_rating_positive_count_todate

- previous equation also applies to clean, service, location, value, room, sleep

- All ratings values (not counts) are on a percentile scale 

- Cities appears to be on area NOT city

- On dates where there are no new reviews available at all, NaNs are placed for all to_date features

## Weekly stats for Paris

**step 1**: building a dataframe that contains all the state columns sampled by week (taking maximum as the weekly state) per hotel id

In [23]:
paris_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354854 entries, 92469 to 776336
Data columns (total 61 columns):
hotel_id                                 354854 non-null object
hotel_name                               354854 non-null object
street                                   354854 non-null object
zip_code                                 354854 non-null object
city                                     354854 non-null object
country                                  354854 non-null object
crawled_at                               354854 non-null object
review_count_state                       354854 non-null int64
average_rating_state                     354854 non-null float64
sleep_rating_state                       66565 non-null float64
location_rating_state                    66582 non-null float64
room_rating_state                        66565 non-null float64
service_rating_state                     66582 non-null float64
value_rating_state                       66565 non-nu

In [24]:
# list of all state columns in paris dataframe

state_cols = [col for col in paris_df if 'state' in col]
print(state_cols)

['review_count_state', 'average_rating_state', 'sleep_rating_state', 'location_rating_state', 'room_rating_state', 'service_rating_state', 'value_rating_state', 'clean_rating_state']


In [25]:
# parsing crawled_at column as datetime

paris_df['crawled_at']=pd.to_datetime(paris_df['crawled_at'])

C:\Users\Dina\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
# for state columns, we should take the maximum value of the week for every hotel id

paris_byhotel_byweek_state = paris_df.groupby(['hotel_id', pd.Grouper(key='crawled_at', freq='W')])[state_cols].max(skipna=True)

In [27]:
paris_byhotel_byweek_state.head()

review_count_state  average_rating_state  \
hotel_id crawled_at                                             
100421   2015-08-23                 346                  80.0   
         2015-08-30                 348                  80.0   
         2015-09-06                 349                  80.0   
         2015-09-13                 352                  80.0   
         2015-09-20                 355                  80.0   

                     sleep_rating_state  location_rating_state  \
hotel_id crawled_at                                              
100421   2015-08-23                90.0                   90.0   
         2015-08-30                90.0                   90.0   
         2015-09-06                90.0                   90.0   
         2015-09-13                90.0                   90.0   
         2015-09-20                90.0                   90.0   

                     room_rating_state  service_rating_state  \
hotel_id crawled_at                                            
100421   2015-08-23               80.0                  80.0   
         2015-08-30               80.0                  80.0   
         2015-09-06               80.0                  80.0   
         2015-09-13               80.0                  80.0   
         2015-09-20               80.0                  80.0   

                     value_rating_state  clean_rating_state  
hotel_id crawled_at                                          
100421   2015-08-23                80.0                90.0  
         2015-08-30                80.0                90.0  
         2015-09-06                80.0                90.0  
         2015-09-13                80.0                90.0  
         2015-09-20                80.0                90.0

In [28]:
paris_byhotel_byweek_state.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53224 entries, (100421, 2015-08-23 00:00:00) to (92741, 2018-04-29 00:00:00)
Data columns (total 8 columns):
review_count_state       53224 non-null int64
average_rating_state     53224 non-null float64
sleep_rating_state       10610 non-null float64
location_rating_state    10614 non-null float64
room_rating_state        10610 non-null float64
service_rating_state     10614 non-null float64
value_rating_state       10610 non-null float64
clean_rating_state       10614 non-null float64
dtypes: float64(7), int64(1)
memory usage: 3.5+ MB


In [29]:
# There are weeks were all the rows for the week for sleep/location... state is NaN so the max is also NaN

test=pd.DataFrame(data=np.array([np.nan, 9, np.nan, np.nan]))
print(test.max(skipna=True))
test_allnan=pd.DataFrame(data=np.array([np.nan, np.nan, np.nan, np.nan]))
print(test_allnan.max(skipna=True))

0    9.0
dtype: float64
0   NaN
dtype: float64


**step 2**: building a dataframe that contains all the count_todate columns sampled by week (taking sum of the week) per hotel id

In [30]:
# list of all count+todate columns in paris dataframe

count_todate_cols = [col for col in paris_df if 'count_todate' in col]
print(count_todate_cols)

['response_rate_negative_count_todate', 'response_rate_neutral_count_todate', 'response_rate_positive_count_todate', 'response_rate_count_todate', 'average_rating_positive_count_todate', 'average_rating_neutral_count_todate', 'average_rating_negative_count_todate', 'average_rating_count_todate', 'sleep_rating_positive_count_todate', 'sleep_rating_neutral_count_todate', 'sleep_rating_negative_count_todate', 'sleep_rating_count_todate', 'location_rating_positive_count_todate', 'location_rating_neutral_count_todate', 'location_rating_negative_count_todate', 'location_rating_count_todate', 'room_rating_positive_count_todate', 'room_rating_neutral_count_todate', 'room_rating_negative_count_todate', 'room_rating_count_todate', 'service_rating_positive_count_todate', 'service_rating_neutral_count_todate', 'service_rating_negative_count_todate', 'service_rating_count_todate', 'value_rating_positive_count_todate', 'value_rating_neutral_count_todate', 'value_rating_negative_count_todate', 'value

In [31]:
# for count_todate columns, we should take the sum value of the week for every hotel id

paris_byhotel_byweek_count_todate = paris_df.groupby(['hotel_id', pd.Grouper(key='crawled_at', freq='W')])[count_todate_cols].sum(skipna=True)

In [32]:
paris_byhotel_byweek_count_todate.head()

response_rate_negative_count_todate  \
hotel_id crawled_at                                        
100421   2015-08-23                                  0.0   
         2015-08-30                                  0.0   
         2015-09-06                                  0.0   
         2015-09-13                                  0.0   
         2015-09-20                                  0.0   

                     response_rate_neutral_count_todate  \
hotel_id crawled_at                                       
100421   2015-08-23                                 0.0   
         2015-08-30                                 0.0   
         2015-09-06                                 0.0   
         2015-09-13                                 1.0   
         2015-09-20                                 1.0   

                     response_rate_positive_count_todate  \
hotel_id crawled_at                                        
100421   2015-08-23                                  0.0   
         2015-08-30                                  2.0   
         2015-09-06                                  1.0   
         2015-09-13                                  4.0   
         2015-09-20                                  1.0   

                     response_rate_count_todate  \
hotel_id crawled_at                               
100421   2015-08-23                         0.0   
         2015-08-30                         2.0   
         2015-09-06                         1.0   
         2015-09-13                         5.0   
         2015-09-20                         2.0   

                     average_rating_positive_count_todate  \
hotel_id crawled_at                                         
100421   2015-08-23                                   0.0   
         2015-08-30                                   2.0   
         2015-09-06                                   1.0   
         2015-09-13                                   4.0   
         2015-09-20                                   1.0   

                     average_rating_neutral_count_todate  \
hotel_id crawled_at                                        
100421   2015-08-23                                  0.0   
         2015-08-30                                  0.0   
         2015-09-06                                  0.0   
         2015-09-13                                  1.0   
         2015-09-20                                  1.0   

                     average_rating_negative_count_todate  \
hotel_id crawled_at                                         
100421   2015-08-23                                   0.0   
         2015-08-30                                   0.0   
         2015-09-06                                   0.0   
         2015-09-13                                   0.0   
         2015-09-20                                   0.0   

                     average_rating_count_todate  \
hotel_id crawled_at                                
100421   2015-08-23                          0.0   
         2015-08-30                          2.0   
         2015-09-06                          1.0   
         2015-09-13                          5.0   
         2015-09-20                          2.0   

                     sleep_rating_positive_count_todate  \
hotel_id crawled_at                                       
100421   2015-08-23                                 0.0   
         2015-08-30                                 1.0   
         2015-09-06                                 0.0   
         2015-09-13                                 3.0   
         2015-09-20                                 0.0   

                     sleep_rating_neutral_count_todate  \
hotel_id crawled_at                                      
100421   2015-08-23                                0.0   
         2015-08-30                                0.0   
         2015-09-06                                0.0   
         2015-09-13                                0.0   
   

In [33]:
paris_byhotel_byweek_count_todate.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53224 entries, (100421, 2015-08-23 00:00:00) to (92741, 2018-04-29 00:00:00)
Data columns (total 32 columns):
response_rate_negative_count_todate      53224 non-null float64
response_rate_neutral_count_todate       53224 non-null float64
response_rate_positive_count_todate      53224 non-null float64
response_rate_count_todate               53224 non-null float64
average_rating_positive_count_todate     53224 non-null float64
average_rating_neutral_count_todate      53224 non-null float64
average_rating_negative_count_todate     53224 non-null float64
average_rating_count_todate              53224 non-null float64
sleep_rating_positive_count_todate       53224 non-null float64
sleep_rating_neutral_count_todate        53224 non-null float64
sleep_rating_negative_count_todate       53224 non-null float64
sleep_rating_count_todate                53224 non-null float64
location_rating_positive_count_todate    53224 non-null float64
location

In [34]:
# There are weeks were all the rows for the week for sleep/location... state is NaN but the sum will be 0 in that case
# which is momentarily not a problem

test=pd.DataFrame(data=np.array([np.nan, 9, np.nan, np.nan]))
print(test.sum(skipna=True))
test_allnan=pd.DataFrame(data=np.array([np.nan, np.nan, np.nan, np.nan]))
print(test_allnan.sum(skipna=True))

0    9.0
dtype: float64
0    0.0
dtype: float64


**step 3**: building a dataframe that contains all the normalized_todate columns sampled by week (taking average of the week) per hotel id

In [35]:
# list of all normalized+todate columns in paris dataframe

normalized_todate_cols = [col for col in paris_df if 'normalized_todate' in col]
print(normalized_todate_cols)

['average_rating_normalized_todate', 'sleep_rating_normalized_todate', 'location_rating_normalized_todate', 'room_rating_normalized_todate', 'service_rating_normalized_todate', 'value_rating_normalized_todate', 'clean_rating_normalized_todate']


In [36]:
# for count_todate columns, we should take the mean value of the week for every hotel id

paris_byhotel_byweek_normalized_todate = paris_df.groupby(['hotel_id', pd.Grouper(key='crawled_at', freq='W')])[normalized_todate_cols].mean()

In [37]:
paris_byhotel_byweek_normalized_todate.head()

average_rating_normalized_todate  \
hotel_id crawled_at                                     
100421   2015-08-23                               NaN   
         2015-08-30                              90.0   
         2015-09-06                              80.0   
         2015-09-13                              80.0   
         2015-09-20                              70.0   

                     sleep_rating_normalized_todate  \
hotel_id crawled_at                                   
100421   2015-08-23                             NaN   
         2015-08-30                            40.0   
         2015-09-06                             0.0   
         2015-09-13                            65.0   
         2015-09-20                             0.0   

                     location_rating_normalized_todate  \
hotel_id crawled_at                                      
100421   2015-08-23                                NaN   
         2015-08-30                                0.0   
         2015-09-06                                0.0   
         2015-09-13                               40.0   
         2015-09-20                                0.0   

                     room_rating_normalized_todate  \
hotel_id crawled_at                                  
100421   2015-08-23                            NaN   
         2015-08-30                           30.0   
         2015-09-06                            0.0   
         2015-09-13                            0.0   
         2015-09-20                            0.0   

                     service_rating_normalized_todate  \
hotel_id crawled_at                                     
100421   2015-08-23                               NaN   
         2015-08-30                              40.0   
         2015-09-06                              80.0   
         2015-09-13                              65.0   
         2015-09-20                               0.0   

                     value_rating_normalized_todate  \
hotel_id crawled_at                                   
100421   2015-08-23                             NaN   
         2015-08-30                             0.0   
         2015-09-06                            80.0   
         2015-09-13                             0.0   
         2015-09-20                             0.0   

                     clean_rating_normalized_todate  
hotel_id crawled_at                                  
100421   2015-08-23                             NaN  
         2015-08-30                             0.0  
         2015-09-06                           100.0  
         2015-09-13                            20.0  
         2015-09-20                             0.0

In [38]:
paris_byhotel_byweek_normalized_todate.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53224 entries, (100421, 2015-08-23 00:00:00) to (92741, 2018-04-29 00:00:00)
Data columns (total 7 columns):
average_rating_normalized_todate     35221 non-null float64
sleep_rating_normalized_todate       35221 non-null float64
location_rating_normalized_todate    35221 non-null float64
room_rating_normalized_todate        35221 non-null float64
service_rating_normalized_todate     35221 non-null float64
value_rating_normalized_todate       35221 non-null float64
clean_rating_normalized_todate       35221 non-null float64
dtypes: float64(7)
memory usage: 3.1+ MB


In [39]:
# There are weeks were all the rows for the week are NaN and the mean will be NaN in that case
# Also, the number of non-null values is equal for all 7 columns because of their connection

test=pd.DataFrame(data=np.array([np.nan, 9, np.nan, np.nan]))
print(test.mean(skipna=True))
test_allnan=pd.DataFrame(data=np.array([np.nan, np.nan, np.nan, np.nan]))
print(test_allnan.mean(skipna=True))

0    9.0
dtype: float64
0   NaN
dtype: float64


In [40]:
# checking data of the first hotel to check work so far

_=paris_df[paris_df['hotel_id']=='100421']
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
     print(_.head(12))

       hotel_id                             hotel_name  \
331713   100421  BEST WESTERN Hotel Villa Des Artistes   
331714   100421  BEST WESTERN Hotel Villa Des Artistes   
331715   100421  BEST WESTERN Hotel Villa Des Artistes   
331716   100421  BEST WESTERN Hotel Villa Des Artistes   
331717   100421  BEST WESTERN Hotel Villa Des Artistes   
331718   100421  BEST WESTERN Hotel Villa Des Artistes   
331719   100421  BEST WESTERN Hotel Villa Des Artistes   
331720   100421  BEST WESTERN Hotel Villa Des Artistes   
331721   100421  BEST WESTERN Hotel Villa Des Artistes   
331722   100421  BEST WESTERN Hotel Villa Des Artistes   
331723   100421  BEST WESTERN Hotel Villa Des Artistes   
331724   100421  BEST WESTERN Hotel Villa Des Artistes   

                              street zip_code   city country crawled_at  \
331713  9 Rue De La Grande Chaumiere    75006  paris      FR 2015-08-19   
331714  9 Rue De La Grande Chaumiere    75006  paris      FR 2015-08-20   
331715  9 Rue De La 

In [41]:
# the sleep/room/... normaized_todate columns have a bug: zeros are placed on dates where there is a non-null value for 
#count_todate columns, but the zeros should NaNs because they do not reflect a value

#easy solution: replace zeros in normalized_to_date columns with NaN, assuming in a 5 stars system on trip-advisor customers 
# CAN NOT give zero starts 
